In [1]:
from llama_index.llms.together import TogetherLLM
from llama_index.core import Settings

llm = TogetherLLM(model="META-LLAMA/LLAMA-3-70B-CHAT-HF", 
api_key="87cac61702f938de91651d60977c8a2978163b011f9ba6e991f0e954b9c2c2ce")
Settings.llm = llm

# separator = "\n",

/home/chandanha/Documents/My_Documents/Excel_data_reader/my_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.readers.file import PandasExcelReader
import importlib
from pathlib import Path

base_reader = PandasExcelReader()
base_docs = base_reader.load_data(Path("Tv_catalogue.xlsx"))

In [3]:
from llama_index.core import SummaryIndex
base_index = SummaryIndex.from_documents([base_docs[0]])
base_query_engine = base_index.as_query_engine()

In [4]:
query_str = "What are all the different Screen Type available"
base_response = base_query_engine.query(query_str)

In [5]:
print("******* PandasExcelReader *******")
print(str(base_response))

******* PandasExcelReader *******
Based on the provided context information, the different screen types available are:

1. LED
2. QLED
3. OLED


In [6]:
Settings.chunk_size = 1024
Settings.chunk_overlap = 128
Settings.context_window = 8100

In [7]:
import nest_asyncio
import os

nest_asyncio.apply()

from llama_index.core.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor,
)
from llama_index.core.node_parser import TokenTextSplitter

text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=1024, chunk_overlap=128
)
title_extractor = TitleExtractor(nodes=5)
qa_extractor = QuestionsAnsweredExtractor(questions=3)

# assume documents are defined -> extract nodes
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[text_splitter, title_extractor, qa_extractor]
)

nodes = pipeline.run(
    documents=base_docs,
    in_place=True,
    show_progress=True,
)


100%|██████████| 3/3 [00:08<00:00,  2.82s/it]


In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
Settings.embed_model = embed_model

In [9]:
from llama_index.core import (
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

# Create a folder for local Qdrant Client
client = qdrant_client.QdrantClient(path="qdrant_excel_tv")

# Create a folder text_collection Qdrant VectorStore
text_store = QdrantVectorStore(client=client, collection_name="text_collection")
storage_context = StorageContext.from_defaults(vector_store=text_store)
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024, chunk_overlap=100, embed_model=embed_model)

# VectorStoreIndex will return index object
index2 = SummaryIndex.from_documents(nodes_no_metadata, storage_context=storage_context)

/tmp/ipykernel_19661/2182097066.py:16: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024, chunk_overlap=100, embed_model=embed_model)


NameError: name 'nodes_no_metadata' is not defined

In [ ]:
from llama_index.core.postprocessor import SentenceTransformerRerank
# Initializing reranking model
rerank = SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=7)

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.prompts import ChatPromptTemplate

chat_text_qa_msgs = [
# Defines the system role message for the chat interaction like guidelines and tone for the AI model.
ChatMessage(
    role=MessageRole.SYSTEM,
    content=("""
        You are a AI Chatbot assistance to answer the question for the excel data i have given.
        Your goal is to answer questions as accurately as possible based on the instructions and context provided.
        After answering to the query at the end tell 'Let me Know if You Have any Queries'
        If the context is Outside the Excel date Don't answer related to the question.
        Just say that, 'The asked question is outside the context of the Excel data, I will answer to the questions related to the excel only'.
        If the context is Outside the Excel Don't try to make up an answer.
        For general questions like 'How are you?' or 'Who are you?', respond accordingly,"""
    ),
),
# Defines the system user message for the chat interaction like query and passing relevent chunks to the AI model.
ChatMessage(
    role=MessageRole.USER,
    content=(
        "Context information is below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Given the context information and not prior knowledge, "
        "answer the question: {query_str} in bullet points or numbered list where appropriate.\n"
    ),
),
]

In [ ]:
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [ ]:
query_engine = index2.as_query_engine(similarity_top_k=3, node_postprocessors=[
                                                            rerank], text_qa_template=text_qa_template)

In [ ]:
user_query = "How can you help me"
response__2 = query_engine.query(user_query)
response__2

In [ ]:
user_query = "List the Tv brands available"
response__2 = query_engine.query(user_query)
response__2

In [ ]:
user_query = "What was the maximum, minimum and average price of the TV"
response__2 = query_engine.query(user_query)
response__2

In [ ]:
user_query = "Which brand has the highest number of TV models listed?"
response__2 = query_engine.query(user_query)
response__2

In [ ]:
user_query = "List all the model name in the TV Brand Panasonic"
response__2 = query_engine.query(user_query)
response__2

In [ ]:
user_query = "List all the model name in the TV Brand Sony"
response__2 = query_engine.query(user_query)
response__2

In [ ]:
user_query = "What is the average price of the TVs listed?"
response__2 = query_engine.query(user_query)
response__2

In [ ]:
user_query = "How many rows are there in the excel"
response__2 = query_engine.query(user_query)
response__2

In [ ]:
user_query = "What is the average price of the TVs listed?"
response__2 = query_engine.query(user_query)
response__2

In [ ]:
user_query = "What is the average price of the TVs listed by considering all the rows?"
response__2 = query_engine.query(user_query)
response__2